In [ ]:
import json

# Read the GPT-4o.json file
with open('GPT-4o.json', 'r') as file:
    data = json.load(file)


import pandas as pd

df = pd.read_csv('Hemonc.csv')
answers = df['answer'].tolist()

In [10]:
len(data)

6212

In [28]:
data[0]['memory'][0]

"To determine the effectiveness of the combination of Cisplatin and Topotecan against the combination of Cisplatin, Topotecan, Hydralazine, and Valproate in the treatment of cervical cancer, let's analyze how these drugs might work together.\n\n1. **Cisplatin**: A platinum-based chemotherapeutic agent known to be effective in treating various cancers, including cervical cancer.\n2. **Topotecan**: A topoisomerase inhibitor used for treating cancers, including cervical cancer. It is often used in combination with cisplatin for enhanced efficacy.\n\nThe combination of Cisplatin and Topotecan is a well-studied regimen for cervical cancer and has shown improved outcomes compared to single-agent therapy.\n\n3. **Hydralazine**: Primarily used as an anti-hypertensive medication, but has been investigated for its role as a DNA demethylating agent, potentially having anticancer effects.\n4. **Valproate (Valproic acid)**: Mainly used as an anticonvulsant, it also possesses properties as a histone

In [23]:
# data[0]
pred = []
for i in data:
    try:
        pred.append(int(i['memory'][0].split('Option ')[1].rstrip('</answer>')[0]))
    except:
        pred.append(0)

# Check the first 10 predictions
pred[:10]

[1, 3, 1, 1, 2, 3, 1, 1, 2, 1]

In [ ]:
import numpy as np

# get rows where pred is not the same as answers and then print the count and only keep those rows in df
# Convert lists to numpy arrays for faster comparison
pred_array = np.array(pred)
answers_array = np.array(answers)

# Verify lengths match
if len(pred_array) != len(answers_array):
    raise ValueError(f"Length mismatch: pred has {len(pred_array)} elements, but answers has {len(answers_array)} elements.")

# Find indices where predictions are wrong
wrong_indices = np.where(pred_array != answers_array)[0]
wrong_count = len(wrong_indices)

print(f"Number of wrong predictions: {wrong_count}")
print(f"Accuracy: {(1 - wrong_count/len(pred_array))*100:.2f}%")

# Create a new DataFrame with wrong predictions more efficiently
wrong_df = df.iloc[wrong_indices].copy()

# Add columns showing the predicted and actual answers
wrong_df['predicted'] = pred_array[wrong_indices]
wrong_df['actual'] = answers_array[wrong_indices]

# Save the wrong predictions to a CSV file
wrong_df.to_csv('wrong_predictions.csv', index=False)

In [55]:

import pandas as pd

df = pd.read_csv('results_full_gemini2flash.csv')
df

,input,model_output,extracted_answer,actual_option,correct
0,{'question': 'Choose an option that best descr...,<answer>Option 2</answer>\n,Option 2,Option 2,True
1,{'question': 'Choose an option that best descr...,<answer>Option 1</answer>\n,Option 1,Option 1,True
2,{'question': 'Choose an option that best descr...,<answer>Option 3</answer>\n,Option 3,Option 1,False
3,{'question': 'Choose an option that best descr...,<answer>Option 3</answer>\n,Option 3,Option 2,False
4,{'question': 'Choose an option that best descr...,<answer>Option 1</answer>\n,Option 1,Option 2,False
...,...,...,...,...,...
6207,{'question': 'Choose an option that best descr...,<answer>Option 1</answer>\n,Option 1,Option 1,True
6208,{'question': 'Choose an option that best descr...,<answer>Option 3</answer>\n,Option 3,Option 2,False
6209,{'question': 'Choose an option that best descr...,<answer>Option 3</answer>\n,Option 3,Option 1,False
6210,{'question': 'Choose an option that best descr...,<answer>Option 2</answer>\n,Option 2,Option 2,True


In [56]:
edf = pd.read_csv('results_full_evidence_gemini2flash.csv')
edf

,input,model_output,extracted_answer,actual_option,correct
0,{'question': 'Choose an option that best descr...,<answer>Option 2</answer>\n,Option 2,Option 2,True
1,{'question': 'Choose an option that best descr...,<answer>Option 1</answer>\n,Option 1,Option 1,True
2,{'question': 'Choose an option that best descr...,<answer>Option 3</answer>\n,Option 3,Option 1,False
3,{'question': 'Choose an option that best descr...,<answer>Option 3</answer>\n,Option 3,Option 2,False
4,{'question': 'Choose an option that best descr...,<answer>Option 2</answer>\n,Option 2,Option 2,True
...,...,...,...,...,...
6207,{'question': 'Choose an option that best descr...,<answer>Option 1</answer>\n,Option 1,Option 1,True
6208,{'question': 'Choose an option that best descr...,<answer>Option 3</answer>\n,Option 3,Option 2,False
6209,{'question': 'Choose an option that best descr...,<answer>Option 3</answer>\n,Option 3,Option 1,False
6210,{'question': 'Choose an option that best descr...,<answer>Option 2</answer>\n,Option 2,Option 2,True


In [57]:
# get a list of true or false values based on the condition that the extracted_answer in edf is equal to the extracted_answer in df
true_false = (edf['extracted_answer'] == df['extracted_answer']).tolist()

# Get unique values and their counts using numpy
unique_values, counts = np.unique(true_false, return_counts=True)
print(dict(zip(unique_values, counts)))

{False: 2127, True: 4085}


In [48]:
# drop all row where 429 is in column extracted_answer

df = df[~df['extracted_answer'].astype(str).str.contains('429')]
df['correct'].value_counts()

correct
True     2628
False     592
Name: count, dtype: int64

In [40]:
df['input'][4]

"{'question': 'Choose an option that best describes the efficacy of Best supportive care compared to Panitumumab monotherapy when used to treat Colorectal cancer (Metastatic).', 'options': ['superior', 'inferior', 'no difference']}"

In [36]:
df['question 1'][0]

'Choose an option that best describes the efficacy of Cisplatin and Topotecan compared to Cisplatin, Topotecan, Hydralazine, Valproate when used to treat Cervical cancer.'

In [32]:
df['input'][0]

"{'question': 'Choose an option that best describes the efficacy of Cisplatin and Topotecan compared to Cisplatin, Topotecan, Hydralazine, Valproate when used to treat Cervical cancer.', 'options': ['superior', 'inferior', 'no difference']}"